In [ ]:
from dflow import SlurmRemoteExecutor

from dflow import upload_artifact, Workflow, Step

from dflow.python import PythonOPTemplate, OP, OPIO, OPIOSign, Artifact, Slices
import os
from typing import List
from pathlib import Path
import subprocess

In [ ]:
from dflow import config
from dflow import s3_config

config["host"] = "https://your-ip-address:60001"
s3_config["endpoint"] = "your-ip-address:60002"

In [ ]:
class CP2KOpt(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({"Opt_input": Artifact(Path)})

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({"Opt_output": Artifact(Path)})

    @OP.exec_sign_check
    def execute(self, op_in: OPIO) -> OPIO:
        cwd = os.getcwd()
        os.chdir(op_in["Opt_input"])
        cmd = "ulimit -s unlimited; srun /opt/cp2k-9.1/cp2k-9.1-Linux-x86_64.ssmp -i input.inp -o output.out"
        subprocess.call(cmd, shell=True)
        os.chdir(cwd)
        return OPIO({"Opt_output": op_in["Opt_input"] / "output.out"})

if you do not have cp2k compiled. Don't worry. They offer an out-of-box version. We can directly use it. Download here: 
However, you need to download the pseudopotential etc. And configure your environment.
....

In [ ]:
slurm_remote_executor = SlurmRemoteExecutor(
    host="your-cluster-address",
    username="your-login-username",
    password="your-login-password",
    header="#!/bin/bash\n#SBATCH --nodes=1\n#SBATCH --ntasks-per-node=4\n#SBATCH --partition=cpu\n#SBATCH -e test.err\n",
)

Structure_Opt = Step(
    "Structure-Opt",
    PythonOPTemplate(CP2KOpt),
    artifacts={"Opt_input": upload_artifact(["./cp2k_opt"])},
    executor=slurm_remote_executor,
)


wf = Workflow(name="cp2k-task")
wf.add(Structure_Opt)
wf.submit()